# Загрузки и библиотеки

Наши данные с электротрицательностью и анизотропиями https://docs.google.com/spreadsheets/d/1SCtFT-960lAdJTIhyI4vf8_Z6oXhFV7uzZMSJzlkmko/edit?usp=sharing

In [1]:
#!pip install rdkit > None #None для того, чтобы не видеть информация по загрузке

In [2]:
from rdkit import Chem

In [3]:
#Чтобы Future warning не портило визуализации
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
import pandas as pd
import numpy as np
import re
from rdkit.Chem.Descriptors import *
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [6]:
# нужные библиотеки
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

In [7]:
df = pd.read_csv(r'C:\Users\Ксения\Project\Megy\content\DATA_2.0.1 - Sheet1.csv')

In [8]:
df

,link,Effective anisotropy constant (J/m3),magnetic anisotropy constant (J/m3),magnetocrystalline anisotropy constant (J/m3),Anisotropy constant,mass_magnetic_susceptibility(emu/Oe*g),"length, nm",Core_composition_formula,magnetization reversibility,magnetocrystalline anisotropy energy (MAE) (J/m3),...,Max/min(size),surface,pol,surf,Magnetic moment of core,LogP of organic coating,Number H acceptors of organic coating,Field amplitude,Field frequency,SAR
0,https://doi.org/10.1016/j.biomaterials.2014.04...,6000,34000,,34000,,14.0,Au1Fe2O3,,...,...,1,,,,2.262,0,0,126,0.32,74
1,https://doi.org/10.1039/d0na00820f ...,,,,1500,,7.1,Co0.03Fe0.97Fe2O4,,...,...,0,,,,2.387,-3.288,7,250,0.11,11.8
2,https://doi.org/10.1039/d0na00820f ...,,,,2000,,6.9,Co0.05Fe0.95Fe2O4,,...,...,0,,,,2.384,-3.288,7,250,0.11,13.3
3,http://dx.doi.org/10.1021/acs.chemmater.5b0478...,,,,82000,,17.0,Co0.1Fe0.9Fe2O4,,...,...,0,,,,2.377,3.212,1,251.37,0.18,430.24
4,http://dx.doi.org/10.1021/acs.chemmater.5b0478...,,,,82000,,17.0,Co0.1Fe0.9Fe2O4,,...,...,0,,,,2.377,3.212,1,201.12,0.3,213.743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,Работа ани ...,62000,223000,,,0.015,2.0,ZnFe2O4,,2670...,...,,,,,,,,,,
564,Работа ани ...,62000,223000,,,0.015,2.5,ZnFe2O4,,2670...,...,,,,,,,,,,
565,Работа ани ...,62000,223000,,,0.015,2.5,ZnFe2O4,,2670...,...,,,,,,,,,,
566,Работа ани ...,62000,223000,,,0.015,5.5,ZnFe2O4,,2670...,...,,,,,,,,,,


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 41 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   link                                                  568 non-null    object 
 1   Effective anisotropy constant (J/m3)                  568 non-null    object 
 2   magnetic anisotropy constant (J/m3)                   568 non-null    object 
 3   magnetocrystalline anisotropy constant (J/m3)         568 non-null    object 
 4   Anisotropy constant                                   568 non-null    object 
 5   mass_magnetic_susceptibility(emu/Oe*g)                568 non-null    object 
 6   length, nm                                            568 non-null    float64
 7   Core_composition_formula                              568 non-null    object 
 8   magnetization reversibility                           568 no

Удалим лишние столбцы с сылками или с отсутсвующими данными

In [10]:
# Удаление пробелов в конце имен столбцов
df.columns = df.columns.str.strip()

# Список столбцов для удаления
cols_to_drop = ['link', 'surface', 'pol', 'surf']

# Удаление столбцов
df.drop(columns=cols_to_drop, inplace=True)

In [11]:
df.columns

Index(['Effective anisotropy constant (J/m3)',
       'magnetic anisotropy constant (J/m3)',
       'magnetocrystalline anisotropy constant (J/m3)', 'Anisotropy constant',
       'mass_magnetic_susceptibility(emu/Oe*g)', 'length, nm',
       'Core_composition_formula', 'magnetization reversibility',
       'magnetocrystalline anisotropy energy (MAE) (J/m3)',
       'Curie temperature (K)', 'exchange constant (J/m) *10^-12',
       'Crystalline', 'interdomain coupling (alpha)', 'pinning loss (A/m)',
       'Effective anisotropy constant (kJ/m3)', 'Spehricity', 'volume',
       'surface area', 'width, nm', 'depth, nm', 'Electronegativity core',
       'Elongation', 'Flatness', 'Space group', 'Demagnetization factors (Nd)',
       'Saturation magnetization (emu/g)', 'Coercivity (Oe)',
       'Remanence magnetization (emu/g)', 'Shell composition (surface)',
       'shape name', 'Max/min(size)', 'Magnetic moment of core',
       'LogP of organic coating', 'Number H acceptors of organic coat

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 37 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Effective anisotropy constant (J/m3)               568 non-null    object 
 1   magnetic anisotropy constant (J/m3)                568 non-null    object 
 2   magnetocrystalline anisotropy constant (J/m3)      568 non-null    object 
 3   Anisotropy constant                                568 non-null    object 
 4   mass_magnetic_susceptibility(emu/Oe*g)             568 non-null    object 
 5   length, nm                                         568 non-null    float64
 6   Core_composition_formula                           568 non-null    object 
 7   magnetization reversibility                        568 non-null    object 
 8   magnetocrystalline anisotropy energy (MAE) (J/m3)  568 non-null    object 
 9   Curie temp

Закодируем параметры типа object: 1.Core и Shell преобразуем в новые колонки с помощью rdkit 2.Crystalline сделаем с помощью 0 и 1, а вот  3. shape - наверное, стоит использовать LabelEncoder

# Анализ данных

In [13]:
df['Crystalline'] = [1 if i == 'yes' else 0 for i in df['Crystalline']]

Теперь формы: сферическая, кубическая и тому подобное закодированы цифрами

In [14]:
df['shape name'].unique()

array(['dimer      ', 'sphere     ', 'cube       ', '           ',
       'rod        ', 'cuboidal   ', 'toroid     ', 'octahedral ',
       'cubic      ', 'polyhedral '], dtype=object)

188 - пропусков в MAE - это уже радует, но больше сократить количество пропусков вручную я не могу

In [15]:
df['magnetocrystalline anisotropy constant (J/m3)'].isnull().sum()

0

Эти данные либо маленькие, либо нулевые:

*   interdomain coupling (magnetoelectric coupling) (mV/cm Oe) - добавлены значения вручную в ноябре
*   pinning loss - добавлены значения в декабре
*   magnetization reversibility

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 37 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Effective anisotropy constant (J/m3)               568 non-null    object 
 1   magnetic anisotropy constant (J/m3)                568 non-null    object 
 2   magnetocrystalline anisotropy constant (J/m3)      568 non-null    object 
 3   Anisotropy constant                                568 non-null    object 
 4   mass_magnetic_susceptibility(emu/Oe*g)             568 non-null    object 
 5   length, nm                                         568 non-null    float64
 6   Core_composition_formula                           568 non-null    object 
 7   magnetization reversibility                        568 non-null    object 
 8   magnetocrystalline anisotropy energy (MAE) (J/m3)  568 non-null    object 
 9   Curie temp

Составим список с названием столбцов на всякий случай

In [17]:
features = ['Effective anisotropy constant (J/m3)', 'magnetic anisotropy constant (J/m3)', 'magnetocrystalline anisotropy constant (J/m3)',
            'Anisotropy constant', 'Core_composition_formula', 'Crystalline', 'magnetocrystalline anisotropy energy (MAE) (J/m3)',
            'Curie temperature (K)', 'Spehricity', 'length, nm', 'exchange constant (J/m) *10^-12',
            'width, nm', 'depth, nm', 'shape name', 'Electronegativity core', 'Elongation', 'Flatness',
            'Space group', 'Demagnetization factors (Nd)', 'Saturation magnetization (emu/g)', 'Coercivity (Oe)',
            'Remanence magnetization (emu/g)', 'Shell composition (surface)',
            'interdomain coupling (alpha)',  'mass_magnetic_susceptibility(emu/Oe*g)']

Данные из колонки Max/min(size) не информативные, так теперь у нас есть колонка формы - правда колонка max/min более заполненная, но я не помню принцип её заполнения

Если бы я помнила можно было бы поступить наоборот, вроде там было что-то про *сферичность, если сфера 0, если нет 1 или наоборот*

In [18]:
df = df.loc[ :, features]
df

,Effective anisotropy constant (J/m3),magnetic anisotropy constant (J/m3),magnetocrystalline anisotropy constant (J/m3),Anisotropy constant,Core_composition_formula,Crystalline,magnetocrystalline anisotropy energy (MAE) (J/m3),Curie temperature (K),Spehricity,"length, nm",...,Elongation,Flatness,Space group,Demagnetization factors (Nd),Saturation magnetization (emu/g),Coercivity (Oe),Remanence magnetization (emu/g),Shell composition (surface),interdomain coupling (alpha),mass_magnetic_susceptibility(emu/Oe*g)
0,6000,34000,,34000,Au1Fe2O3,0,...,,,14.0,...,1,1,167,0.333333333,28,3.564,19.253,...,,
1,,,,1500,Co0.03Fe0.97Fe2O4,0,...,,1,7.1,...,1,1,227,0.333333333,20,0,0,carboxymethylcellulose ...,,
2,,,,2000,Co0.05Fe0.95Fe2O4,0,...,,1,6.9,...,1,1,227,0.333333333,20,0,0,carboxymethylcellulose ...,,
3,,,,82000,Co0.1Fe0.9Fe2O4,0,...,,1,17.0,...,1,1,227,0.333333333,73.5,143,10.8,decanoic acid ...,,
4,,,,82000,Co0.1Fe0.9Fe2O4,0,...,,1,17.0,...,1,1,227,0.333333333,86.2,9.1,69,decanoic acid ...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,62000,223000,,,ZnFe2O4,0,2670...,600,,2.0,...,1,1,227,0.333333333,31,74.7,0.5,...,,0.015
564,62000,223000,,,ZnFe2O4,0,2670...,600,,2.5,...,1,1,227,0.333333333,33.7,66.9,0.6,...,,0.015
565,62000,223000,,,ZnFe2O4,0,2670...,600,,2.5,...,1,1,227,0.333333333,64.3,37.8,1.8,...,,0.015
566,62000,223000,,,ZnFe2O4,0,2670...,600,,5.5,...,1,1,227,0.333333333,85.1,100.7,9.9,...,,0.015


Надо исправить ошибку кодировки данных кристалличности. Получается, что все пропуски автоматически заполняются нулями


```
list_ = []

for i in df['Crystalline']:
    if i != '':
        if i == 'no':
            list_.append(0)
        else:
            list_.append(1)
    else:
        list_.append(None)
```
Не получилось, не понимаю, почему в список не получается добавить пустые значения



Посмотрим количнство пропусков в столбцах

In [19]:
#df['Crystalline'] = [1 if i == 'yes' else 0 if pd.notna(i) else np.nan for i in df['Crystalline']]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 25 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Effective anisotropy constant (J/m3)               568 non-null    object 
 1   magnetic anisotropy constant (J/m3)                568 non-null    object 
 2   magnetocrystalline anisotropy constant (J/m3)      568 non-null    object 
 3   Anisotropy constant                                568 non-null    object 
 4   Core_composition_formula                           568 non-null    object 
 5   Crystalline                                        568 non-null    int64  
 6   magnetocrystalline anisotropy energy (MAE) (J/m3)  568 non-null    object 
 7   Curie temperature (K)                              568 non-null    object 
 8   Spehricity                                         568 non-null    object 
 9   length, nm

In [21]:
df = df.rename(columns = {'Shell composition (surface)': 'shell_comp'})

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 25 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Effective anisotropy constant (J/m3)               568 non-null    object 
 1   magnetic anisotropy constant (J/m3)                568 non-null    object 
 2   magnetocrystalline anisotropy constant (J/m3)      568 non-null    object 
 3   Anisotropy constant                                568 non-null    object 
 4   Core_composition_formula                           568 non-null    object 
 5   Crystalline                                        568 non-null    int64  
 6   magnetocrystalline anisotropy energy (MAE) (J/m3)  568 non-null    object 
 7   Curie temperature (K)                              568 non-null    object 
 8   Spehricity                                         568 non-null    object 
 9   length, nm

Чтобы получить данные из структуры ядра и кожуры, нужно добавить столбец с элементами из которых она состоит и стехиометрию. Сначала заполним эти столбцы нулями

In [23]:
df.insert(0, 'Elements', 0)
df.insert(0, 'Stoichiometry', 0)

In [24]:
for i, row in df.iterrows():
    df.Elements[i] = re.findall('[A-Z][a-z]?', df.Core_composition_formula[i])
    df.Stoichiometry[i] = re.findall('[\d.]+', df.Core_composition_formula[i])
    print(df.Elements[i])
    print(df.Stoichiometry[i])

for i, row in df.iterrows():
        r = df.Elements[i]
        for elem in range(len(r) - 1):
            if r[elem] == 'Fe' and r[elem + 1] == 'Fe':
                r[elem] = 'Fe2+' # Fe2+
            elif r[elem] == 'Fe':
                 r[elem] = 'Fe3+' # Fe3+
        print(r)
        df.Elements[i] = r

['Au', 'Fe', 'O']
['1', '2', '3']
['Co', 'Fe', 'Fe', 'O']
['0.03', '0.97', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.05', '0.95', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.1', '0.9', '2', '4']
['Co', 'Mn', 'Fe', 'Fe', 'O']
['0.24', '0.15', '0.61', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.3', '0.7', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'Fe', 'O']
['0.5', '0.5', '2', '4']
['Co', 'Fe', 'F

C:\Users\Ксения\AppData\Local\Temp\ipykernel_25708\660171926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Elements[i] = re.findall('[A-Z][a-z]?', df.Core_composition_formula[i])
C:\Users\Ксения\AppData\Local\Temp\ipykernel_25708\660171926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Stoichiometry[i] = re.findall('[\d.]+', df.Core_composition_formula[i])
C:\Users\Ксения\AppData\Local\Temp\ipykernel_25708\660171926.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Co', 'Fe3+', 'O']
['Cu', 'Fe3+', 'O']
['Cu', 'Fe3+', 'O']
['Cu', 'Fe3+', 'O']
['Cu', 'Cu', 'Fe3+', 'O']
['Cu', 'Cu', 'Fe3+', 'O']
['Cu', 'Cu', 'Fe3+', 'O']
['Cu', 'Cu', 'Fe3+', 'O']
['Fe3+', 'Mn', 'Fe3+', 'O']
['Fe3+', 'Mn', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe3+', 'O']
['Fe2+', 'Fe

In [25]:
for i, row in df.iterrows():
    df.Stoichiometry[i] = list(map(float, re.findall('[\d.]+', df.Core_composition_formula[i])))
    print(df.Stoichiometry[i])

[1.0, 2.0, 3.0]
[0.03, 0.97, 2.0, 4.0]
[0.05, 0.95, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.1, 0.9, 2.0, 4.0]
[0.24, 0.15, 0.61, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.3, 0.7, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.5, 0.5, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 0.3, 2.0, 4.0]
[0.7, 

C:\Users\Ксения\AppData\Local\Temp\ipykernel_25708\3301359452.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Stoichiometry[i] = list(map(float, re.findall('[\d.]+', df.Core_composition_formula[i])))


[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.0, 4.0]
[1.0, 2.

Сделаем тоже самое для поверхности

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 27 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Stoichiometry                                      568 non-null    object 
 1   Elements                                           568 non-null    object 
 2   Effective anisotropy constant (J/m3)               568 non-null    object 
 3   magnetic anisotropy constant (J/m3)                568 non-null    object 
 4   magnetocrystalline anisotropy constant (J/m3)      568 non-null    object 
 5   Anisotropy constant                                568 non-null    object 
 6   Core_composition_formula                           568 non-null    object 
 7   Crystalline                                        568 non-null    int64  
 8   magnetocrystalline anisotropy energy (MAE) (J/m3)  568 non-null    object 
 9   Curie temp

Нужно ли делать это для покрытия, если количетсво пропусков для вещества покрытия больше 50%

Добавим колонку общего спина и посчитаем спин для каждой наночастицы отдельно

In [27]:
df.insert(0, 'common_spin', 0)

In [28]:
df = df.astype({'common_spin' : float})

Составим словарь спинов всех элементов, что у нас есть

In [29]:
dic_spin = { 'Fe2+': 2, 'Fe3+': 5/2, 'Cu': 1/2, 'Zn': 0, 'Mn': 5/2, 'Co': 2,
             'Ni': 3/2, 'O': 0, 'Si': 0, 'Fe': 2, '': '',
             'Au': 1/2, 'C': 1/2, 'Mg': 0, 'Pd': 0 }

In [30]:
pd.options.mode.chained_assignment = None #чтобы убрать ошибку SettingWithCopyWarning

In [31]:
df.Elements

0            [Au, Fe3+, O]
1      [Co, Fe2+, Fe3+, O]
2      [Co, Fe2+, Fe3+, O]
3      [Co, Fe2+, Fe3+, O]
4      [Co, Fe2+, Fe3+, O]
              ...         
563          [Zn, Fe3+, O]
564          [Zn, Fe3+, O]
565          [Zn, Fe3+, O]
566          [Zn, Fe3+, O]
567          [Zn, Fe3+, O]
Name: Elements, Length: 568, dtype: object

In [32]:
for i, _ in df.iterrows():
    sum_spin = 0

    # Проверьте, что Core_composition_formula[i] не равно 0, не пусто и имеет длину > 0
    if df.Core_composition_formula[i] != 0 and df.Core_composition_formula[i] != [] and len(df.Core_composition_formula[i]) != 0:

        # Убедитесь, что Elements[i] существует и является списком
        if not df.Elements[i] or not isinstance(df.Elements[i], list):
            continue

        # Перебираем элементы в Elements[i]
        for elem in df.Elements[i]:
            # Проверьте, что индекс элемента elem существует в Elements[i]
            if elem in df.Elements[i] and df.Elements[i].index(elem) < len(df.Stoichiometry[i]):
                ind = df.Elements[i].index(elem)
                sum_spin += df.Stoichiometry[i][ind] * dic_spin.get(elem, 0)  # Используем get() для безопасного доступа к dic_spin

        # Проверка на деление на ноль перед присвоением значения common_spin
        total_stoichiometry = sum(df.Stoichiometry[i])
        if total_stoichiometry != 0:
            df.at[i, 'common_spin'] = sum_spin / total_stoichiometry
        else:
            df.at[i, 'common_spin'] = 0 #nan


In [33]:
df['common_spin'].unique()

array([0.91666667, 1.        , 1.01071429, 2.33333333, 0.78571429,
       0.33333333, 0.53571429, 1.17647059, 1.08333333, 1.05263158,
       1.07142857, 0.83333333, 1.38888889, 1.03571429, 0.96428571,
       0.92857143, 0.89285714, 1.66666667, 2.5       , 0.82142857,
       0.86170213, 0.75465839, 0.67127072, 0.60447761, 1.70807453,
       2.16666667, 0.        , 1.5       , 0.93714286, 0.85714286,
       0.91428571, 0.75      , 0.71428571])

In [34]:
df.isna().sum() * 100 / len(df)

common_spin                                          0.0
Stoichiometry                                        0.0
Elements                                             0.0
Effective anisotropy constant (J/m3)                 0.0
magnetic anisotropy constant (J/m3)                  0.0
magnetocrystalline anisotropy constant (J/m3)        0.0
Anisotropy constant                                  0.0
Core_composition_formula                             0.0
Crystalline                                          0.0
magnetocrystalline anisotropy energy (MAE) (J/m3)    0.0
Curie temperature (K)                                0.0
Spehricity                                           0.0
length, nm                                           0.0
exchange constant (J/m) *10^-12                      0.0
width, nm                                            0.0
depth, nm                                            0.0
shape name                                           0.0
Electronegativity core         

In [35]:
df.insert(0, 'org_coating_LogP', 0), df.insert(0, 'org_coating_HAcceptors', 0)
df = df.astype({'org_coating_LogP': float, 'org_coating_HAcceptors': float})

In [36]:
LogP = {'Sodium citrate': 'C(C(=O)[O-])C(CC(=O)[O-])(C(=O)[O-])O',
        'Citric acid': 'OC(=O)CC(O)(C(=O)O)CC(=O)O',
        'DSPE': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'Alendronate': 'O=P(O)(O)C(O)(CCCN)P(=O)(O)O',
        'PMAO-PEG': 'CCCCCCCCCCCCCCCCC=C.C1=CC(=O)OC1=O',
        'PEG': 'OCCO', 'DHCA': 'OC(=O)CCc1ccc(O)c(O)c1',
        'HDA-G2': 'CCCCCCCCCCCCCCCCN', 'DMSA': 'O=C(O)[C@@H](S)[C@@H](S)C(=O)O',
        'PEG-350': 'OCCO', 'PEG-550': 'OCCO', 'PEG-1100': 'OCCO', 'PEG-2000': 'OCCO',
        'RGD': 'C1C(=O)NC(C(=O)NC(C(=O)NC(C(=O)NC(C(=O)N1)CCCN=C(N)N)CCCCN)CC2=CC=C(C=C2)O)CC(=O)O.C(=O)O.C(F)(F)F.C(=O)(C(F)(F)F)O',
        'poly(isobutylenemaleic) acid anhydride': 'CC(C)C(=O)OC(=O)C(C)C', 'Cysteamine': 'C(CS)N', 'PVP-10': 'O=C1N(C=C)CCC1',
        'PEG-phospholipid': 'CCCCCCCCCCCCCCCC(=O)OCC(COP(=O)([O-])OCC[N+](C)(C)C)OC(=O)CCCCCCCCCCCCCCC',
        'Oleic acid': 'CCCCCCCC\C=C/CCCCCCCC(O)=O', 'DSPE-PEG5k': 'CCCCCCCCC=CCCCCCCCC(=O)OCC(COP(=O)(O)OCCN)OC(=O)CCCCCCCC=CCCCCCCCC',
        'PCL5k-b-PEG5k': 'C1CCC(=O)OCC1', 'PEG-PMMA': 'CC(=C)C(=O)OC', 'DP-PEG': '[O-]P(=O)([O-])OP(=O)([O-])[O-]',
        'CC-PEG': 'Oc1c(O)cccc1', 'HX-PEG': 'CC(=O)NO',
        'DSPE-mPEG1000': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'DSPE-mPEG550': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'DSPE-mPEG750': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'DSPE-mPEG2000': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'DSPE-mPEG5000': 'CCCCCCCCCCCCCCCCCC(O[C@@H](COP(OCCN)(O)=O)COC(CCCCCCCCCCCCCCCCC)=O)=O',
        'Tween-80': 'CCCCCCCCC=CCCCCCCCC(=O)OCCOCC(C1C(C(CO1)OCCO)OCCO)OCCO', 'PEI': 'NCCN',
        'citrate': 'C(C(=O)[O-])C(CC(=O)[O-])(C(=O)[O-])O', 'GA(Gallic Acid)-PEG': 'C1=C(C=C(C(=C1O)O)O)C(=O)O',
        'poly- (maleic anhydride alt-1-octadecene)': 'CCCCCCCCCCCCCCCCC=C.C1=CC(=O)OC1=O',
        'citrate anions': 'C(C(=O)[O-])C(CC(=O)[O-])(C(=O)[O-])O',
        'decanoic acid': 'CCCCCCCCCC(=O)O', 'oleic acid- coated': 'CCCCCCCC\C=C/CCCCCCCC(O)=O',
        'carboxymethylcellulose': 'CC(=O)O.C(C(C(C(C(C=O)O)O)O)O)O',
        'Folic Acid': 'C1=CC(=CC=C1C(=O)NC(CCC(=O)O)C(=O)O)NCC2=CN=C3C(=N2)C(=O)N=C(N3)N',
        'Curcumin': 'COC1=C(C=CC(=C1)C=CC(=O)CC(=O)C=CC2=CC(=C(C=C2)O)OC)O',
        'phosphatidylcholine': 'CCCCCCCCCCCCCCCCCCCCCCCC(=O)OC(COC(=O)CCCCCCCC=CCCCCCC)COP(=O)([O-])OCC[N+](C)(C)C',
		'oleic acid': 'CCCCCCCC\C=C/CCCCCCCC(O)=O', 'DSPE-PEG5k': 'CCCCCCCCC=CCCCCCCCC(=O)OCC(COP(=O)(O)OCCN)OC(=O)CCCCCCCC=CCCCCCCCC',
		'chitosan': 'COC(=O)NC1C(C(C(OC1OC2C(OC(C(C2O)N)OC3C(OC(C(C3O)N)O)CO)CO)CO)OC4C(C(C(C(O4)CO)OC5C(C(C(C(O5)CO)OC6C(C(C(C(O6)CO)OC7C(C(C(C(O7)CO)OC8C(C(C(C(O8)CO)OC9C(C(C(C(O9)CO)O)O)N)O)N)O)N)O)N)O)N)O)N)O',
		'citric acid': 'OC(=O)CC(O)(C(=O)O)CC(=O)O', 'folate chitosan': 'C1=CC(=CC=C1C(=O)NC(CCC(=O)O)C(=O)O)NCC2=CN=C3C(=N2)C(=O)NC(=N3)N',
        'N-(trimethoxysilylpropyl)-ethylenediaminetriacetate': 'C(CN(CCN(CC(=O)O)CC(=O)O)CC(=O)O)C[Si](O)(O)O', 'triethylene glycol': 'C(COCCOCCO)O',
		'meso-2,3-dimercaptosuccinic acid': 'O=C(O)[C@@H](S)[C@@H](S)C(=O)O',
		'TMAH':'C[N+](C)(C)C.[OH-]', 'poly(2-hydroxyethyl aspartamide)':'C(CO)NC(=O)C(CC(=O)N)N', 'citric acid': 'C(C(=O)O)C(CC(=O)O)(C(=O)O)O',
		'Pluronic F-127': 'CC(COCCO)OCCO', 'DTPMP':'C(CN(CP(=O)(O)O)CP(=O)(O)O)N(CCN(CP(=O)(O)O)CP(=O)(O)O)CP(=O)(O)O', 'Chitosan':'COC(=O)NC1C(C(C(OC1OC2C(OC(C(C2O)N)OC3C(OC(C(C3O)N)O)CO)CO)CO)OC4C(C(C(C(O4)CO)OC5C(C(C(C(O5)CO)OC6C(C(C(C(O6)CO)OC7C(C(C(C(O7)CO)OC8C(C(C(C(O8)CO)OC9C(C(C(C(O9)CO)O)O)N)O)N)O)N)O)N)O)N)O)N)O',
		'Gly': 'C(C(=O)O)N', 'Lys':'C(CCN)CC(C(=O)O)N', 'Trp': 'C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N', 'polyaspartic acid': 'C(C(C(=O)O)N)C(=O)O',
		'PVA': 'C=COC(C)=O'}

In [37]:
for i, _ in df.iterrows(): #перебираем строки таблицы
    if df.shell_comp[i] in LogP.keys(): #условие на то, что наше покрытие органическое
        m = Chem.MolFromSmiles(LogP[df.shell_comp[i]]) #вытаскиваем молекулу из SMILES представления, используя модуль rdkit
        df.org_coating_HAcceptors[i] = NumHAcceptors(m) #считаем для неё число акцепторов водорода
        df.org_coating_LogP[i] = MolLogP(m) #и липофильность

In [38]:
df.org_coating_HAcceptors.unique()

array([0.])

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 30 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   org_coating_HAcceptors                             568 non-null    float64
 1   org_coating_LogP                                   568 non-null    float64
 2   common_spin                                        568 non-null    float64
 3   Stoichiometry                                      568 non-null    object 
 4   Elements                                           568 non-null    object 
 5   Effective anisotropy constant (J/m3)               568 non-null    object 
 6   magnetic anisotropy constant (J/m3)                568 non-null    object 
 7   magnetocrystalline anisotropy constant (J/m3)      568 non-null    object 
 8   Anisotropy constant                                568 non-null    object 
 9   Core_compo

Пересмотрим описание морфологии частицы, чтобы избежать сильного влияния этих параемтров на друг друга

In [40]:
df.insert(0, 'volume', 0)
df.insert(0, 'area', 0)
df = df.astype({'volume': float, 'area': float})

In [41]:
df['shape name'].unique()

array(['dimer      ', 'sphere     ', 'cube       ', '           ',
       'rod        ', 'cuboidal   ', 'toroid     ', 'octahedral ',
       'cubic      ', 'polyhedral '], dtype=object)

In [42]:
df = df.rename(columns = {'shape name': 'shape_name'})


Можно было бы использовать для вычисления объема и площади поверхности



In [43]:
for i, row in df.iterrows(): #перебираем построчно
  #если мы имеем околосферические частицы или материал очень странной формы (Nanoflower), или значение формы пропущено,
  #то используем формулу подсчета объема сферы:
    if df.shape_name[i] in ['spherical', 'sphere', 'Nanoflower', 'Spherical', 'sphere', 'Micelle',
                            'Sphere on sphere', 'sperical', 'Sandwich-like', 'Yolk-shell', 'spherical'] or str(df.shape_name[i]) == 'nan': #условие
       df.volume[i] = 4.19 * float(row['length, nm']) ** 3 #подсчет объема
       df.area[i] = 12.57 * float(row['length, nm']) ** 2 #площадь
      #если мы имеем частицы с формой параллелепипеда то используем соответствующую формулу для подсчета:
    elif df.shape_name[i] in ['Plate', 'cuboidal', 'dimer', "Cubic on the cubic's corners", 'Cubic',
                              'Nanowhiskers', 'cubic', 'cube', 'Rod', 'rod', 'Rhombohedron']: #формула
       df.volume[i] = float(row['length, nm']) * float(row['length, nm']) * float(row['width, nm']) #непосредственно подсчет объема
       df.area[i] = 2 * float(row['length, nm']) * float(row['depth, nm']) + 4 * float(row['length, nm']) * float(row['width, nm'])
    elif df.shape_name[i] in ['Hexaplate', 'hexagon']: #аналогично для гексагональных частиц
       df.volume[i] = 2.60 * float(row['length, nm'])**2 * float(row['width, nm'])
       df.area[i] = 2 * 2.60 * float(row['length, nm'])**2 + 6 * float(row['length, nm']) * float(row['width, nm'])
    elif df.shape_name[i] in ['Octapod', 'octopod', 'Octapode', 'octahedral']:
       df.volume[i] = 0.47 * float(row['length, nm'])**3
       df.area[i] = 3.46 * float(row['length, nm'])**2
    elif df.shape_name[i] in ['Tetrahedron', 'tetrahedron', 'polyhedral']: #и для тетраэдров
       df.volume[i] = 0.118 * float(row['length, nm'])**3
       df.area[i] = 1.73 * float(row['length, nm'])**2

In [44]:
df

,area,volume,org_coating_HAcceptors,org_coating_LogP,common_spin,Stoichiometry,Elements,Effective anisotropy constant (J/m3),magnetic anisotropy constant (J/m3),magnetocrystalline anisotropy constant (J/m3),...,Elongation,Flatness,Space group,Demagnetization factors (Nd),Saturation magnetization (emu/g),Coercivity (Oe),Remanence magnetization (emu/g),shell_comp,interdomain coupling (alpha),mass_magnetic_susceptibility(emu/Oe*g)
0,0.0,0.0,0.0,0.0,0.916667,"[1.0, 2.0, 3.0]","[Au, Fe3+, O]",6000,34000,,...,1,1,167,0.333333333,28,3.564,19.253,...,,
1,0.0,0.0,0.0,0.0,1.000000,"[0.03, 0.97, 2.0, 4.0]","[Co, Fe2+, Fe3+, O]",,,,...,1,1,227,0.333333333,20,0,0,carboxymethylcellulose ...,,
2,0.0,0.0,0.0,0.0,1.000000,"[0.05, 0.95, 2.0, 4.0]","[Co, Fe2+, Fe3+, O]",,,,...,1,1,227,0.333333333,20,0,0,carboxymethylcellulose ...,,
3,0.0,0.0,0.0,0.0,1.000000,"[0.1, 0.9, 2.0, 4.0]","[Co, Fe2+, Fe3+, O]",,,,...,1,1,227,0.333333333,73.5,143,10.8,decanoic acid ...,,
4,0.0,0.0,0.0,0.0,1.000000,"[0.1, 0.9, 2.0, 4.0]","[Co, Fe2+, Fe3+, O]",,,,...,1,1,227,0.333333333,86.2,9.1,69,decanoic acid ...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,0.0,0.0,0.0,0.0,1.666667,"[2.0, 4.0]","[Zn, Fe3+, O]",62000,223000,,...,1,1,227,0.333333333,31,74.7,0.5,...,,0.015
564,0.0,0.0,0.0,0.0,1.666667,"[2.0, 4.0]","[Zn, Fe3+, O]",62000,223000,,...,1,1,227,0.333333333,33.7,66.9,0.6,...,,0.015
565,0.0,0.0,0.0,0.0,1.666667,"[2.0, 4.0]","[Zn, Fe3+, O]",62000,223000,,...,1,1,227,0.333333333,64.3,37.8,1.8,...,,0.015
566,0.0,0.0,0.0,0.0,1.666667,"[2.0, 4.0]","[Zn, Fe3+, O]",62000,223000,,...,1,1,227,0.333333333,85.1,100.7,9.9,...,,0.015


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 32 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   area                                               568 non-null    float64
 1   volume                                             568 non-null    float64
 2   org_coating_HAcceptors                             568 non-null    float64
 3   org_coating_LogP                                   568 non-null    float64
 4   common_spin                                        568 non-null    float64
 5   Stoichiometry                                      568 non-null    object 
 6   Elements                                           568 non-null    object 
 7   Effective anisotropy constant (J/m3)               568 non-null    object 
 8   magnetic anisotropy constant (J/m3)                568 non-null    object 
 9   magnetocry

In [46]:
df['volume']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
563    0.0
564    0.0
565    0.0
566    0.0
567    0.0
Name: volume, Length: 568, dtype: float64

ТУТ

In [47]:
df.insert(0, 'total_magnetization', 0)
#полная намагниченность
#total_magnetization = (Saturation magnetization (emu/g) * voluem)/(9.27*10**-21)

# Преобразование столбца 'Saturation magnetization (emu/g)' в числовой формат
df['Saturation magnetization (emu/g)'] = pd.to_numeric(df['Saturation magnetization (emu/g)'], errors='coerce')

# Проверка типов данных в столбце 'Saturation magnetization (emu/g)'
print(df['Saturation magnetization (emu/g)'].dtype)

#df['total_magnetization'] = df.apply(lambda row: (row['Saturation magnetization (emu/g)'] * row['volume']) / (9.27*10**-21) if not pd.isnull(row['Saturation magnetization (emu/g)']) and not pd.isnull(row['volume']) else np.nan, axis=1)
# Функция для вычисления total_magnetization
def calculate_total_magnetization(row):
    """
    Generates the total magnetization for nanoparticles

    The Total magnetization is product of saturation magnetization of particle and volume of particle divided  into constant(Mb = 9.27*10^-21)
    
    Args: row(list of int)

    Returns: total magneization or nan if some data is not found
    
    Raises:
            TypeError if row hasn't int ot float objects
            ValueError: Input contains NaN, infinity or a value too large for dtype('float64')

    Examples:
            >>>calculate_total_magnetization(5)
            3.89542*10^25 
    """
    if not pd.isnull(row['Saturation magnetization (emu/g)']) and not pd.isnull(row['volume']):
        if isinstance(row['Saturation magnetization (emu/g)'], (int, float)) and isinstance(row['volume'], (int, float)):
            return (row['Saturation magnetization (emu/g)'] * row['volume']) / (9.27*10**-21)
        else:
            return np.nan #need thing about it 
    else:
        return np.nan

# Применение функции к каждой строке и добавление нового столбца
df['total_magnetization'] = df.apply(calculate_total_magnetization, axis=1)
df['total_magnetization']

float64


0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
563    0.0
564    0.0
565    0.0
566    0.0
567    NaN
Name: total_magnetization, Length: 568, dtype: float64

In [48]:
df['area']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
563    0.0
564    0.0
565    0.0
566    0.0
567    0.0
Name: area, Length: 568, dtype: float64

# Посмотрим на распредение данных и на количество пропусков

In [49]:
df.isna().sum() * 100 / len(df)

total_magnetization                                  0.880282
area                                                 0.000000
volume                                               0.000000
org_coating_HAcceptors                               0.000000
org_coating_LogP                                     0.000000
common_spin                                          0.000000
Stoichiometry                                        0.000000
Elements                                             0.000000
Effective anisotropy constant (J/m3)                 0.000000
magnetic anisotropy constant (J/m3)                  0.000000
magnetocrystalline anisotropy constant (J/m3)        0.000000
Anisotropy constant                                  0.000000
Core_composition_formula                             0.000000
Crystalline                                          0.000000
magnetocrystalline anisotropy energy (MAE) (J/m3)    0.000000
Curie temperature (K)                                0.000000
Spehrici

Можно заполнить пропуски в колонках, где пропусков не более 15% от общего числа данных

Нужно не забыть убрать повторение столбца interdomain coupling - в документе с данными, есть только один такой столбец, но в colab их два. Оказалось, что это не ошибка исходного документа


# Заполнение пропусков KNN

In [50]:
nan_before_knn = df.isna().sum().sum()
print(f"Number of NaN values before KNN imputation: {nan_before_knn}")

Number of NaN values before KNN imputation: 10


Список колонк, в которых есть пропуски

In [51]:
colomn_turn = ['area', 'volume', 'org_coating_HAcceptors',
               'Effective anisotropy constant (J/m3)', 'magnetic anisotropy constant (J/m3)',
               'magnetocrystalline anisotropy constant (J/m3)', 'Anisotropy constant',
               'Curie temperature (K)', 'exchange constant (J/m) *10^-12', 'magnetocrystalline anisotropy energy (MAE) (J/m3)',
               'Electronegativity core', 'Space group', 'Demagnetization factors (Nd)',
               'Saturation magnetization (emu/g)', 'Coercivity (Oe)',
               'Remanence magnetization (emu/g)', 'interdomain coupling (alpha)',
               'mass_magnetic_susceptibility(emu/Oe*g)']

Название колонки, которую можно почистить и можно поменять, а также процент данных, которые нужно удалить можно уменьшить или увлечить в по желанию

In [52]:
# Указываем имя столбца и процент удаления
column_name = 'Effective anisotropy constant (J/m3)'
delete_percent = 0.5

# Создаем рабочий датафрейм
df_for_kNN = df.copy()
df_for_kNN = df_for_kNN.drop(columns=['Core_composition_formula', 'shell_comp', 'shape_name', 'Stoichiometry', 'Elements'])

# Рандомное заполнение всех пропущенных значений
imputer = KNNImputer(n_neighbors=5)
imputed = imputer.fit_transform(df_for_kNN)
df_for_kNN = pd.DataFrame(imputed, columns=df_for_kNN.columns)

# Нормализация данных
scaled_df = StandardScaler().fit_transform(df_for_kNN)

# df_for_kNN['cluster'] = kmeans.labels_  # Убираем связанные с кластеризацией строки
# df['cluster'] = kmeans.labels_  # Убираем связанные с кластеризацией строки

# num_in_each_clust = df_for_kNN.copy()

# dict_overall = {}
# for i, row in num_in_each_clust.iterrows():
#     dict_overall[num_in_each_clust.cluster[i]] = dict_overall.get(num_in_each_clust.cluster[i], 0) + 1

# num_nan = df.copy()
# for i, row in num_nan.iterrows():
#     if str(num_nan[column_name][i]) != 'nan':
#         num_nan = num_nan.drop(i)

# dict_nan = {}
# for i, row in num_nan.iterrows():
#     dict_nan[num_nan.cluster[i]] = dict_nan.get(num_nan.cluster[i], 0) + 1

# newnew = pd.DataFrame(columns=num_nan.columns.tolist())

# for clust in dict_nan:
#     new1 = num_nan.copy()
#     for i, _ in num_nan.iterrows():
#         if num_nan.cluster[i] != clust:
#             new1 = new1.drop(i)

#     remove_n = int(dict_nan[clust] * delete_percent)
#     if remove_n != 0:
#         drop_indices = np.random.choice(new1.index, remove_n, replace=False)
#         new1_subset = new1.drop(drop_indices)
#     else:
#         continue

#     newnew = newnew.append(new1_subset)

df_decreased = df.copy()

# for i in df.index:
#     if str(df[column_name][i]) == 'nan' and i not in newnew.index:
#         df_decreased = df_decreased.drop(i)

print(f'После удаления некоторых пропущенных значений число пропусков равно \n {df_decreased.isna().sum() * 100 / len(df_decreased)}')
df_decreased.reset_index(drop=True, inplace=True)

df1 = df_decreased.copy()
df1 = df1.drop(columns=['Core_composition_formula', 'shell_comp', 'shape_name', 'Stoichiometry', 'Elements'])

imputer = KNNImputer(n_neighbors=4)
imputed = imputer.fit_transform(df1)
df_imputed = pd.DataFrame(imputed, columns=df1.columns)

new = pd.concat([df[['Core_composition_formula', 'shell_comp', 'shape_name', 'Stoichiometry', 'Elements']].reindex(df_imputed.index), df_imputed], axis=1)
print(f' Проверим, все ли заполнилось \n {new.isna().sum() * 100 / len(new)}')
print()
print(f'До удаления пропусков было {len(df)} элементов')
print(f'После стало {len(new)} элементов')

ValueError: could not convert string to float: '                                     '

Как видно Nan значения в первых двух колонках, которыее далее не используются

In [ ]:
nan_after_knn = new.isna().sum().sum()
print(f"Number of NaN values after KNN imputation: {nan_after_knn}")

In [ ]:
nan_after_knn = new.isna().sum()
print(f"Number of NaN values after KNN imputation: {nan_after_knn}")

Как видим пропусков теперь нет, но элементов осталось 529 из 568

In [ ]:
new.info()

In [ ]:
df = new.copy()

Сохраним копию изменений в формате csv

In [ ]:
df

# Корреляция

In [ ]:
sns.set (font_scale= 2)
corr_matrix = df.corr()
mask = np.zeros_like(corr_matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(50, 55))
heatmap = sns.heatmap(corr_matrix, mask=mask, square=True, linewidths=0.5,
					  cmap=sns.color_palette("coolwarm", as_cmap=True), cbar_kws = {'shrink': 0.6, 'ticks': [-1, -0.5, 0, 0.5, 1]}, #для изменения цвета
												   vmin = -1, vmax = 1, annot = True, annot_kws ={'size': 20}) #annot=False удалит данные из DATA
plt.show()

Посмотрим корреляцию данных, как можно заметить больше всего накладываются друг на друга данные связанные с геометрическими параметрами частиц, так как у нас уже есть объём и площадь частицы, может можно отбросиь эти данные

Создадим на их основе параметры длина/ширину или плозадь на объём

In [ ]:
df.info()

In [ ]:
df.insert(0, 'Max/min(size)', 0)
df.insert(0, 'Area/volume', 0)

In [ ]:
df = df.astype({'Max/min(size)': float, 'Area/volume': float})

In [ ]:
df = df.rename(columns = {'length, nm': 'length', 'width, nm': 'width', 'depth, nm': 'depth'})

In [ ]:
for i, _ in df.iterrows():
    maxim = np.max(np.array([df.length[i], df.width[i], df.depth[i]])) #берем максимальное значение из трех
    minim = np.min(np.array([df.length[i], df.width[i], df.depth[i]])) #берем минимальное значение из трех
    df['Max/min(size)'][i] = maxim/minim
    df['Area/volume'][i] = df.area[i]/df.volume[i] #находим отношение объема к площади

In [ ]:
df['Area/volume'].unique()

In [ ]:
cols_to_drop = ['length', 'width', 'depth', 'shape_name', 'shell_comp']
df.drop(columns=cols_to_drop, inplace=True)

Вновь проверим корреляцию между получившимися дескриптоорами

In [ ]:
del df['Elongation']
del df['Flatness']

In [ ]:
del df['area']
del df['volume']

In [ ]:
sns.set (font_scale= 2)
corr_matrix = df.corr()
mask = np.zeros_like(corr_matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(30, 35))
heatmap = sns.heatmap(corr_matrix, mask=mask, square=True, linewidths=0.5,
					  cmap=sns.color_palette("coolwarm", as_cmap=True), cbar_kws = {'shrink': 0.6, 'ticks': [-1, -0.5, 0, 0.5, 1]},
												   vmin = -1, vmax = 1, annot = True, annot_kws ={'size': 20}) #annot=False удалит данные из DATA
plt.title("Корреляционная матрица всех параметров")
plt.show()

# Визуализации

In [ ]:
df['Core_composition_formula'].unique()

Как видно многие формулы были записаны некоректно, поэтому исправим их немного

In [ ]:
df.info()

Для удобства переименуем столбец с 'magnetocrystalline anisotropy constant (J/m3)' на 'MAE'

In [ ]:
df = df.rename(columns = {'magnetocrystalline anisotropy energy (MAE) (J/m3)': 'MAE'})

Надо как-то это исправить

In [ ]:
ax = df.groupby(['Core_composition_formula']).sum().plot(kind='pie', y='MAE', autopct='%1.0f%%')
ax.legend(bbox_to_anchor = (2.1, 1.25), loc = 'best')
plt.title("Количество разных наночастиц")
plt.show()

In [ ]:
cols_to_drop = ['Core_composition_formula', 'Stoichiometry', 'Elements']
df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
df['Area/volume'] = df['Area/volume'].fillna(df['Area/volume'].mean())

In [ ]:
nan_after = df.isna().sum()
print(f"Number of NaN values : {nan_after}")

In [ ]:
df.to_csv('O_nan.csv', index=False, float_format='%.3f')

In [ ]:
for i in df.columns:
  df[[i]] = np.log1p(df[[i]])

In [ ]:
# Выбор более устойчивого метода нормализации
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

for i in df.columns:
  df[[i]] = scaler.fit_transform(df[[i]])

In [ ]:
columns = df.columns
sns.set(style="whitegrid")

# Создание графиков распределения для каждого столбца
for column in df.columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(df[column], kde=True, color='skyblue', edgecolor='black')
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Density')
    plt.show()

In [ ]:
columns = df.columns

# Создание графиков распределения для каждого столбца
for column in columns:
    plt.figure(figsize=(8, 6))
    plt.hist(df[column], bins=30, color='skyblue', edgecolor='black')  # Можно настроить количество бинов
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [ ]:
df.info()

Видно, что у нас получилось бимодальное распределение по температуре Кюри с пиками при температуре 500 K и 900 K

In [ ]:
plt.title("Распредление температуры Кюри")
sns.histplot(df['Curie temperature (K)'], kde=True)

Существует большой перекос данных в сторону неклассифицированных или некристалических

In [ ]:
sns.countplot(x='Crystalline', data=df, palette='hls')
plt.title("Количество кристаллических и некристаллических")

# MinMaxScaler

Уберём NaN значения

Перевёдем данные в дипазон от 0 до 1, то есть нормализуем их

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
df.columns.tolist()

Для красоты поменяем названия

In [ ]:
col = ['Area/volume', 'Max/min(size)', 'org_coating_HAcceptors',
       'org_coating_LogP', 'common_spin', 'Effective anisotropy constant (J/m3)',
       'magnetic anisotropy constant (J/m3)', 'MAE', 'Anisotropy constant ',
       'Crystalline', 'Curie temperature (K)', 'Spehricity',
       'exchange constant (J/m) *10^-12', 'Electronegativity core', 'Space group',
       'Demagnetization factors (Nd)', 'Saturation magnetization (emu/g)', 'Coercivity (Oe)',
       'Remanence magnetization (emu/g)', 'interdomain coupling (alpha)', 'mass_magnetic_susceptibility(emu/Oe*g)']

for i in col:
  df[i] = scaler.fit_transform(df[[i]])

In [ ]:
df = df.rename(columns = {'Effective anisotropy constant (J/m3)': 'Eff_anisotropy_const'})
df = df.rename(columns = {'Curie temperature (K)': 'Curie temperature'})
df = df.rename(columns = {'exchange constant (J/m) *10^-12': 'Exchange_constant'})
df = df.rename(columns = {'Demagnetization factors (Nd)': 'Demagnetization_factors'})
df = df.rename(columns = {'Saturation magnetization (emu/g)': 'Saturation_magnetization'})
df = df.rename(columns = {'Max/min(size)': 'Max/min'})
df = df.rename(columns = {'Coercivity (Oe)': 'Coercivity'})
df = df.rename(columns = {'Remanence magnetization (emu/g)': 'Remanence_magnetization'})
df = df.rename(columns = {'interdomain coupling (alpha)': 'Interdomain_coupling'})
df = df.rename(columns = {'mass_magnetic_susceptibility(emu/Oe*g)': 'Mass_magnetic_susceptibility'})
df = df.rename(columns = {'magnetic anisotropy constant (J/m3)': 'Magnetic anisotropy'})
df = df.rename(columns = {'magnetocrystalline anisotropy constant (J/m3)': 'Magnetocrystalline_anisotropy'})

In [ ]:
columns = df.columns
sns.set(style="whitegrid")

# Создание графиков распределения для каждого столбца
for column in df.columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(df[column], kde=True, color='skyblue', edgecolor='black')
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Density')
    plt.show()

In [ ]:
df.to_csv ('Doom.csv', index= False, float_format='%.3f')

In [ ]:
print(df.columns)

Видно, что значения MAE и magnetic anisotropy comstamt имеют выбросы и больше всего значений находятся в диапазоне от 0 до 2

In [ ]:
df.boxplot(column=['Magnetic anisotropy', 'MAE'], grid=False, color='blue')

In [ ]:
df

Посмотрим расположение пропусков в столбцах

In [ ]:
nan_after = df.isna().sum()
print(f"Number of NaN values : {nan_after}")

Построим Violin плоты всех величин, чтобы посмотреть на распределение каждой величины, Min/Max - категориальный параметр, как и Crystalline

In [ ]:
k = 1
f, ax = plt.subplots(figsize=(50, 100))
for i in df.loc[:, 'Area/volume': 'Mass_magnetic_susceptibility'].columns.to_list():
      plt.subplot(5, 5, k)
      matplotlib.rcParams['axes.spines.right'] = False
      matplotlib.rcParams['axes.spines.top'] = False
      sns.violinplot(df[i], color='#630ee3', set_linewidth=100)
      plt.tick_params(axis='both', labelsize=50, width=10)
      plt.xlabel(f'{i}', fontsize=50)
      k += 1
      plt.tight_layout()
plt.tight_layout()

In [ ]:
df.describe()

Mx/min и space group - имеет категориальный характер нужно поменять вид графика,а interdomain coupling не несёт никакой нагрузки

In [ ]:
columns_of_interest = df.columns[df.columns.get_loc('Area/volume'): df.columns.get_loc('Mass_magnetic_susceptibility') + 1]

num_columns = len(columns_of_interest)
num_rows = num_columns // 5 + (num_columns % 5 > 0)  # Определение количества строк

plt.figure(figsize=(50, 100))

for idx, column in enumerate(columns_of_interest, start=1):
    plt.subplot(num_rows, 5, idx)
    sns.violinplot(x=df[column], color='#630ee3')
    plt.title(column, fontsize=50)
    plt.tick_params(axis='both', labelsize=40)
    plt.tight_layout()

plt.show()

Надо переделать отображения 'Max//min', 'Space group', 'Crystalline'

In [ ]:
columns_of_interest = df.columns[df.columns.get_loc('Area/volume'): df.columns.get_loc('Mass_magnetic_susceptibility') + 1]

num_columns = len(columns_of_interest)
num_rows = num_columns // 5 + (num_columns % 5 > 0)  # Определение количества строк

plt.figure(figsize=(50, 100))

for idx, column in enumerate(columns_of_interest, start=1):
  plt.subplot(num_rows, 5, idx)
  sns.violinplot(x=df[column], color='#630ee3')
  plt.title(column, fontsize=50)
  plt.tick_params(axis='both', labelsize=40)
  plt.tight_layout()

plt.show()

Сохраним полученный датасет

In [ ]:
df

In [ ]:
columns = df.columns
sns.set(style="whitegrid")

# Создание графиков распределения для каждого столбца
for column in df.columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(df[column], kde=True, color='skyblue', edgecolor='black')
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Density')
    plt.show()

In [ ]:
df.to_csv('/content/FINal.csv', index=False, float_format='%.3f') #сохраняем финальный вариант нашего датасета